In [2]:
len

<function len(obj, /)>

In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
 chunk_size=1000,
 chunk_overlap=20,
 length_function=len,
 is_separator_regex=False,
)
loader = DirectoryLoader('Hindi-Aesthetics-Corpus/Corpus', loader_cls=TextLoader)
docs = loader.load_and_split(text_splitter=text_splitter)

In [4]:
import pandas as pd
data = []
for doc in docs:
 row_data = {
 "page_content": doc.page_content,
 "metadata": doc.metadata
 }
 data.append(row_data)
df = pd.DataFrame(data)
df['page_content'] = df['page_content'].replace('\\n', ' ', regex=True)
import fasttext as ft
# Loding model for Hindi.
embed_model = ft.load_model('wiki.hi.bin')

In [5]:
df['embeddings'] = df['page_content'].apply(lambda x: (embed_model.get_sentence_vector(x)).tolist())


In [6]:
df['id'] = range(1, len(df) + 1)


In [7]:
payload = df[['page_content', 'metadata']].to_dict(orient='records')


In [12]:
len(df['embeddings'][0])

300

In [41]:
from qdrant_client import QdrantClient
client = QdrantClient(host="localhost", port=6333)

In [42]:
# from qdrant_client import QdrantClient
# client = QdrantClient(location=':memory:')

In [43]:
from qdrant_client.models import VectorParams, Distance, Batch
client.delete_collection(collection_name="my_collection")
client.recreate_collection(
   collection_name="my_collection",
   vectors_config=VectorParams(size=300, distance=Distance.COSINE),
)

True

In [44]:
client.upsert(
 collection_name="my_collection",
 points=Batch(
     ids=df['id'].to_list()[:10],
     payloads=payload[:10],
     vectors=df['embeddings'].to_list()[:10],
 ),
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [32]:
import numpy as np
from qdrant_client.models import PointStruct

vectors = np.random.rand(100, 300)
client.upsert(
   collection_name="my_collection",
   points=[
      PointStruct(
            id=idx,
            vector=vector.tolist(),
            payload={"color": "red", "rand_number": idx % 10}
      )
      for idx, vector in enumerate(vectors)
   ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)